In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from matplotlib import font_manager, rc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
font_path = "c:\WINDOWS\Fonts\GULIM.TTC"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rc('font', size=8) 

In [2]:
dataframe = pd.read_csv(r'C:\Users\hyssk\AICE_\open\train.csv')
dataframe.drop(columns='ID',inplace=True)
dataframe

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [3]:
dataframe['근로기간']=dataframe['근로기간'].replace('1 years','1 year')
dataframe['근로기간']=dataframe['근로기간'].replace('3','3 years')
dataframe['근로기간']=dataframe['근로기간'].replace('<1 year','< 1 year')
dataframe['근로기간']=dataframe['근로기간'].replace('10+years','10+ years')
dataframe['근로기간'].value_counts()

10+ years    32481
2 years       8450
< 1 year      8144
3 years       7670
1 year        6305
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
Name: 근로기간, dtype: int64

In [10]:
X=dataframe[['대출금액','대출기간', '연간소득', '부채_대비_소득_비율', '총계좌수', '총상환원금', '총상환이자']]
y=dataframe[['대출등급']]

In [14]:
from sklearn.preprocessing import RobustScaler, LabelEncoder

rs=RobustScaler()

for col in ['대출금액','연간소득','부채_대비_소득_비율','총계좌수','총상환원금','총상환이자']:
    X[col]=rs.fit_transform(X[[col]])
X

c:\Users\hyssk\anaconda3\envs\pytorch_project\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\hyssk\anaconda3\envs\pytorch_project\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\hyssk\anaconda3\envs\pytorch_project\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,총상환원금,총상환이자
0,-0.313043,36 months,-0.108696,0.012413,-0.600000,-0.799589,-0.658871
1,-0.173913,60 months,0.956522,0.278510,-0.200000,-0.299830,-0.121543
2,-0.347826,36 months,0.326087,-0.786656,-0.666667,0.442737,-0.310055
3,-0.173913,36 months,0.978261,-0.283165,-0.600000,-0.363706,-0.307383
4,0.086957,60 months,-0.113478,0.515904,-0.333333,-0.493852,-0.316915
...,...,...,...,...,...,...,...
96289,-0.173913,36 months,2.391304,-0.730023,0.600000,0.504190,0.470992
96290,0.869565,60 months,0.978261,-1.053530,0.066667,-0.018686,1.304132
96291,-0.173913,36 months,0.108696,-0.581846,-0.133333,1.192545,-0.105069
96292,-0.086957,36 months,-0.211413,-0.111715,-0.200000,1.044372,1.219174


In [28]:
le=LabelEncoder()
X['대출기간']=le.fit_transform(X[['대출기간']]).reshape(-1,1)
le2=LabelEncoder()
y=pd.DataFrame(le2.fit_transform(y).reshape(-1,1),columns=['대출등급'])
y

c:\Users\hyssk\anaconda3\envs\pytorch_project\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\hyssk\anaconda3\envs\pytorch_project\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,대출등급
0,2
1,1
2,0
3,2
4,1
...,...
96289,2
96290,4
96291,0
96292,3


In [29]:
X

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,총상환원금,총상환이자
0,-0.313043,0,-0.108696,0.012413,-0.600000,-0.799589,-0.658871
1,-0.173913,1,0.956522,0.278510,-0.200000,-0.299830,-0.121543
2,-0.347826,0,0.326087,-0.786656,-0.666667,0.442737,-0.310055
3,-0.173913,0,0.978261,-0.283165,-0.600000,-0.363706,-0.307383
4,0.086957,1,-0.113478,0.515904,-0.333333,-0.493852,-0.316915
...,...,...,...,...,...,...,...
96289,-0.173913,0,2.391304,-0.730023,0.600000,0.504190,0.470992
96290,0.869565,1,0.978261,-1.053530,0.066667,-0.018686,1.304132
96291,-0.173913,0,0.108696,-0.581846,-0.133333,1.192545,-0.105069
96292,-0.086957,0,-0.211413,-0.111715,-0.200000,1.044372,1.219174


In [30]:
y

,대출등급
0,2
1,1
2,0
3,2
4,1
...,...
96289,2
96290,4
96291,0
96292,3


# 데이터 분할


In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y)
X_train

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,총상환원금,총상환이자
84054,0.778261,0,0.978261,-0.170675,0.800000,2.772427,1.335344
87781,-0.347826,0,0.543478,0.328161,0.866667,-0.387305,-0.553030
83487,0.869565,1,1.413043,-0.169899,0.800000,0.401849,2.682314
42170,1.217391,0,1.304348,-0.279286,0.800000,-0.799589,-0.658871
49021,-0.228261,1,-0.869565,0.591156,-0.600000,-0.799589,-0.658871
...,...,...,...,...,...,...,...
19157,-0.347826,0,-0.543478,-0.017843,-0.400000,-0.427214,-0.317686
6884,1.826087,1,0.141304,-0.404189,-0.600000,-0.086191,0.139036
53651,-0.904348,0,0.760870,-0.931730,-0.066667,-0.799589,-0.658871
90175,1.339130,1,0.108696,1.477890,0.333333,-0.335645,0.510441


In [40]:
y_train

,대출등급
84054,1
87781,0
83487,4
42170,1
49021,6
...,...
19157,2
6884,1
53651,2
90175,4


In [46]:
model=Sequential([
    Dense(4,input_shape=(7,)),
    Dense(8,activation='relu'),
    Dense(16,activation='relu'),
    Dense(7,activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(X_train,y_train, epochs= 50,validation_data=(X_test,y_test))

Epoch 1/50
2107/2107 [==============================] - 2s 868us/step - loss: 1.2656 - accuracy: 0.4956 - val_loss: 1.0645 - val_accuracy: 0.5973
Epoch 2/50
2107/2107 [==============================] - 2s 812us/step - loss: 0.9195 - accuracy: 0.6624 - val_loss: 0.8023 - val_accuracy: 0.7139
Epoch 3/50
2107/2107 [==============================] - 2s 931us/step - loss: 0.7368 - accuracy: 0.7372 - val_loss: 0.6762 - val_accuracy: 0.7594
Epoch 4/50
2107/2107 [==============================] - 2s 831us/step - loss: 0.6444 - accuracy: 0.7718 - val_loss: 0.6215 - val_accuracy: 0.7648
Epoch 5/50
2107/2107 [==============================] - 2s 818us/step - loss: 0.6033 - accuracy: 0.7866 - val_loss: 0.6073 - val_accuracy: 0.7754
Epoch 6/50
2107/2107 [==============================] - 2s 800us/step - loss: 0.5879 - accuracy: 0.7935 - val_loss: 0.5724 - val_accuracy: 0.7986
Epoch 7/50
2107/2107 [==============================] - 2s 797us/step - loss: 0.5780 - accuracy: 0.7996 - val_loss: 0.5688 -